In [ ]:
import optuna
import xgboost as xgb
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score

# --- 1. Preparazione Dati ---
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# --- 2. Definizione della "Objective Function" ---
# Questa è la funzione che Optuna chiamerà centinaia di volte.
# Deve restituire UN solo numero: il valore da ottimizzare (es. Accuracy).

def objective(trial):
    # A. Suggerimento Parametri (Search Space)
    # Optuna sceglie i valori in modo intelligente basandosi sulla storia passata
    params = {
        'objective': 'binary:logistic',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000), # Intero tra 100 e 1000
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True), # Scala logaritmica
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0), # % feature per albero
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Reg (Lasso)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Reg (Ridge)
        'tree_method': 'hist',
        'device': 'gpu',
        'n_jobs': -1,
        'random_state': 42,
        # Early Stopping nel costruttore per versioni recenti (o gestito in fit)
    }
    
    
    # B. Creazione Modello
    model = xgb.XGBClassifier(**params)
    
    # C. Validazione Robusta (Cross-Validation)
    # Usiamo 5-Fold per essere sicuri che il risultato non sia fortuna
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    
    # Restituiamo la media dell'accuratezza
    return scores.mean()

# --- 3. Creazione dello Studio ---
print("--- Inizio Ottimizzazione Bayesiana ---")
# direction='maximize' perché vogliamo ALTA accuratezza (se fosse RMSE useremmo minimize)
study = optuna.create_study(direction='maximize', study_name="XGBoost_BreastCancer")

# Lanciamo l'ottimizzazione (es. 50 tentativi)
# Optuna mostrerà una progress bar con i risultati in tempo reale
study.optimize(objective, n_trials=50, show_progress_bar=True)

# --- 4. Risultati ---
print("\n--- Risultati Ottimizzazione ---")
print(f"Miglior Trial (Tentativo #{study.best_trial.number})")
print(f"Accuratezza Migliore (CV): {study.best_value:.4f}")
print("Migliori Iperparametri:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# --- 5. Validazione Finale sul Test Set ---
# Ricostruiamo il modello con i parametri vincenti
best_params = study.best_params
final_model = xgb.XGBClassifier(**best_params, n_jobs=-1, random_state=42)
final_model.fit(X_train, y_train)
final_acc = final_model.score(X_test, y_test)

print(f"\nAccuratezza Finale sul Test Set: {final_acc:.4f}")

[I 2026-01-06 19:51:12,218] A new study created in memory with name: XGBoost_BreastCancer


--- Inizio Ottimizzazione Bayesiana ---


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2026-01-06 19:51:12,974] Trial 0 finished with value: 0.9670329670329669 and parameters: {'n_estimators': 431, 'learning_rate': 0.046126194303366744, 'max_depth': 9, 'subsample': 0.9826452144281044, 'colsample_bytree': 0.5997488722531131, 'reg_alpha': 5.588442879808919e-05, 'reg_lambda': 5.423099576094846}. Best is trial 0 with value: 0.9670329670329669.
[I 2026-01-06 19:51:14,660] Trial 1 finished with value: 0.9604395604395604 and parameters: {'n_estimators': 549, 'learning_rate': 0.005540913426134001, 'max_depth': 10, 'subsample': 0.5903986727471358, 'colsample_bytree': 0.9211118105074912, 'reg_alpha': 0.00011764987664583178, 'reg_lambda': 8.430663132589746e-07}. Best is trial 0 with value: 0.9670329670329669.
[I 2026-01-06 19:51:15,920] Trial 2 finished with value: 0.956043956043956 and parameters: {'n_estimators': 574, 'learning_rate': 0.0028135694225538636, 'max_depth': 5, 'subsample': 0.5433970779139308, 'colsample_bytree': 0.6711770492473093, 'reg_alpha': 0.00044924748984913

In [1]:
import optuna
import xgboost as xgb
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from optuna.integration import XGBoostPruningCallback
from sklearn.metrics import log_loss

# --- 1. Preparazione Dati ---
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_vali, y_train, y_vali = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# --- 2. Definizione della "Objective Function" ---
# Questa è la funzione che Optuna chiamerà centinaia di volte.
# Deve restituire UN solo numero: il valore da ottimizzare (es. Accuracy).

def objective(trial):
    # A. Suggerimento Parametri (Search Space)
    # Optuna sceglie i valori in modo intelligente basandosi sulla storia passata
    params = {
        'objective': 'binary:logistic',
        'n_estimators': 4000,
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True), # Scala logaritmica
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0), # % feature per albero
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Reg (Lasso)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Reg (Ridge)
        'tree_method': 'hist',
        'device': 'gpu',
        'n_jobs': -1,
        'random_state': 42,
        # Early Stopping nel costruttore per versioni recenti (o gestito in fit)
        'earlystopping': 100
    }
    
    # B. Creazione Modello
    model = xgb.XGBClassifier(**params)
    
    model.fit(
        X_train,y_train,
        eval_set=[(X_vali, y_vali)],
        eval_metric="logloss",
        verbose=False,
        callbacks=[
            XGBoostPruningCallback(trial, "validation_0-logloss")
        ]
    )

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    return log_loss(y_val, y_pred_proba)
    

# --- 3. Creazione dello Studio ---
print("--- Inizio Ottimizzazione Bayesiana ---")
# direction='maximize' perché vogliamo ALTA accuratezza (se fosse RMSE useremmo minimize)
study = optuna.create_study(direction='maximize', study_name="XGBoost_BreastCancer")

# Lanciamo l'ottimizzazione (es. 50 tentativi)
# Optuna mostrerà una progress bar con i risultati in tempo reale
study.optimize(objective, n_trials=50, show_progress_bar=True)

# --- 4. Risultati ---
print("\n--- Risultati Ottimizzazione ---")
print(f"Miglior Trial (Tentativo #{study.best_trial.number})")
print(f"Accuratezza Migliore (CV): {study.best_value:.4f}")
print("Migliori Iperparametri:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# --- 5. Validazione Finale sul Test Set ---
# Ricostruiamo il modello con i parametri vincenti
best_params = study.best_params
final_model = xgb.XGBClassifier(**best_params, n_jobs=-1, random_state=42)
final_model.fit(X_train, y_train)
final_acc = final_model.score(X_test, y_test)

print(f"\nAccuratezza Finale sul Test Set: {final_acc:.4f}")

[I 2026-01-06 20:41:25,782] A new study created in memory with name: XGBoost_BreastCancer


--- Inizio Ottimizzazione Bayesiana ---


  0%|          | 0/50 [00:00<?, ?it/s]

[W 2026-01-06 20:41:25,794] Trial 0 failed with parameters: {'learning_rate': 0.28754560680450264, 'max_depth': 5, 'subsample': 0.5470549602394306, 'colsample_bytree': 0.9676794290860993, 'reg_alpha': 8.38699124585968, 'reg_lambda': 2.277376003105444e-06} because of the following error: TypeError("XGBClassifier.fit() got an unexpected keyword argument 'eval_metric'").
Traceback (most recent call last):
  File "c:\Users\m-rog\anaconda3\Lib\site-packages\optuna\study\_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\m-rog\AppData\Local\Temp\ipykernel_41976\1584032117.py", line 43, in objective
    model.fit(
    ~~~~~~~~~^
        X_train,y_train,
        ^^^^^^^^^^^^^^^^
    ...<5 lines>...
        ]
        ^
    )
    ^
  File "c:\Users\m-rog\anaconda3\Lib\site-packages\xgboost\core.py", line 774, in inner_f
    return func(**kwargs)
TypeError: XGBClassifier.fit() got an unexpected keyword argument 'eval_metric'
[W 2026-01-06 20:41:25,796] Trial

TypeError: XGBClassifier.fit() got an unexpected keyword argument 'eval_metric'